In [27]:
# =========================================
# VWAPクロス × ACFゲート（ALT単体の自己相関/ACFで環境判定）バックテスト
# 追加要件対応版：
#  - 最大DD: 最高含み益からではなく、エントリー価格からの最大含み損（MAE, %）
#  - ワントレードの最大損失（実現損失の最小値）
#  - ACFゲート通過中はVWAPクロス瞬間でなくてもエントリー許可
#  - 自己相関閾値を下回ったらExitする機能をON/OFF切替
#  - ACFゲート通過中に逆向きクロスの場合はドテン（基本機能）
#  - 損切り（クールダウン無し）
# =========================================

# 0) Libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# 1) Strategy / BT Parameters  ----------------------------------------------
# データ
ALT_csv = 'Market_Bybit_ASTERUSDT_1min_20250901-20250926'
time_col = "timestamp"

# ACF/自己相関 設定
ACF_WINDOW = 60             # ACFのローリング窓（分）
ACF_LAGS = [1, 2, 3]        # 参照するラグ
ACF_MIN_LAG1 = 0.15         # ラグ1の下限（環境ONの最低ライン）
ACF_MIN_MEAN = 0.10         # 選択ラグ平均の下限
ACF_PERSIST = 1             # 条件継続本数（1=一発、2=2本連続）
ACF_SMOOTH_HALFLIFE = 5    # ACFの平滑化（EWMA）半減期（バー数）
EXIT_ON_ACF_DROP = False     # True: ACFが閾値割れで即時Exit / False: 維持

# VWAP
VWAP_SHORT = 7
VWAP_MEDIUM = 14

# BT
INITIAL_CAPITAL = 1000.0
TAKER_FEE = 0.00035         # 片道手数料 (例: 0.035%)
NOTIONAL_SIZE = 500.0       # 名目USD
MIN_QTY = 0.0               # 取引所の最小数量（無ければ0）
STOP_LOSS_PCT = 0.05        # 損切り閾値（小数：2%なら0.02）。0以下で無効

# 2) IO & Prep  ------------------------------------------------------------
def load_market(csv, rename_prefix=None):
    df = pd.read_csv(f'01.data/{csv}.csv')
    df[time_col] = pd.to_datetime(df[time_col])
    df = df.sort_values(time_col)
    if rename_prefix:
        rename = {c: f"{c}_{rename_prefix}" for c in ["O","H","L","C","V"] if c in df.columns}
        df = df.rename(columns=rename)
    return df

# ALTデータのみ（この版ではBTCは不要）
df_altOHLCV = load_market(ALT_csv, "ALT")       # timestamp, O_ALT/H_ALT/L_ALT/C_ALT/V_ALT

# 3) ALTの自己相関/ACFを算出  --------------------------------------------

def rolling_acf(series: pd.Series, lag: int, window: int) -> pd.Series:
    """系列とそのラグずらしとのローリング相関（=自己相関）を返す。"""
    return series.rolling(window, min_periods=window).corr(series.shift(lag))


def compute_acf_features(df_alt_close: pd.DataFrame) -> pd.DataFrame:
    x = df_alt_close[[time_col, "C_ALT"]].copy()
    # 1分logリターン
    x["ret_ALT"] = np.log(x["C_ALT"]).diff()

    # ACF（ラグごと）
    for k in ACF_LAGS:
        acf_k = rolling_acf(x["ret_ALT"], lag=k, window=ACF_WINDOW)
        # 平滑化（推奨：推定のノイズ低減）
        x[f"acf_lag{k}"] = acf_k.ewm(halflife=ACF_SMOOTH_HALFLIFE, adjust=False).mean()

    # 代表量：ラグ1、選択ラグ平均
    lag_cols = [f"acf_lag{k}" for k in ACF_LAGS]
    x["acf_mean_sel"] = x[lag_cols].mean(axis=1)

    # ACFゲート（この列をBTへ渡してExit判定にも使う）
    cond_lag1 = x["acf_lag1"] > ACF_MIN_LAG1
    cond_mean = x["acf_mean_sel"] > ACF_MIN_MEAN
    gate_basic = cond_lag1 & cond_mean
    x["acf_gate_basic"] = gate_basic
    x["acf_gate"] = gate_basic.rolling(ACF_PERSIST, min_periods=ACF_PERSIST).sum() == ACF_PERSIST

    return x[[time_col, "ret_ALT"] + lag_cols + ["acf_mean_sel", "acf_gate", "acf_gate_basic"]]

acf_df = compute_acf_features(df_altOHLCV[[time_col, "C_ALT"]])

# 4) ALT側でVWAPクロス & ACFゲート合成  ---------------------------------

def add_vwap_and_signal(df_alt: pd.DataFrame, acf_df: pd.DataFrame) -> pd.DataFrame:
    d = df_alt.merge(acf_df, on=time_col, how="left").copy()

    # VWAP
    d["Typical_price"] = (d["O_ALT"] + d["H_ALT"] + d["L_ALT"] + d["C_ALT"]) / 4
    d["TPxV"] = d["Typical_price"] * d["V_ALT"]

    roll_tpv_s = d["TPxV"].rolling(VWAP_SHORT,  min_periods=1).sum()
    roll_v_s   = d["V_ALT"].rolling(VWAP_SHORT,  min_periods=1).sum()
    roll_tpv_m = d["TPxV"].rolling(VWAP_MEDIUM, min_periods=1).sum()
    roll_v_m   = d["V_ALT"].rolling(VWAP_MEDIUM, min_periods=1).sum()

    d["VWAP_short"]  = roll_tpv_s / roll_v_s
    d["VWAP_medium"] = roll_tpv_m / roll_v_m

    # クロス Regime（1: long bias / 0: short bias）
    d["Regeme_vwap"] = (d["VWAP_short"] > d["VWAP_medium"]).astype(int)
    reg_prev = d["Regeme_vwap"].shift(1)
    reg_changed = (d["Regeme_vwap"] != reg_prev).fillna(False)

    # 点灯バー（参考・デバッグ用）
    d["Trade_ignittion_vwap"] = np.where(reg_changed, d["Regeme_vwap"], np.nan)

    # === ACFゲート中は“常時”エントリー許可 ===
    #   ゲート通過中は、VWAPクロス瞬間でなくても現在のRegeme方向でシグナルを出す
    d["Trade"] = np.where(d["acf_gate"], d["Regeme_vwap"], np.nan)

    # 参考：フィルター無し版（比較用）
    d["Trade_vwap_only"] = np.where(d["Trade_ignittion_vwap"].notna(), d["Regeme_vwap"], np.nan)
    return d


df_all = add_vwap_and_signal(df_altOHLCV, acf_df)

# 5) バックテスト  ---------------------------------------------------------

def run_backtest(
    df_in: pd.DataFrame,
    initial_capital: float = INITIAL_CAPITAL,
    fee: float = TAKER_FEE,
    notional_size: float = NOTIONAL_SIZE,
    min_qty: float = MIN_QTY,
    exit_on_acf_drop: bool = EXIT_ON_ACF_DROP,
    stop_loss_pct: float = STOP_LOSS_PCT,
) -> pd.DataFrame:
    """名目サイズ固定・線形PnL・片道手数料・ドテン対応
    追加: MAE(最大含み損, %) と ワントレードの最大実現損失、損切りカウント
    - ACFゲート外れ時に exit_on_acf_drop=True なら即時クローズ
    - ACFゲート内でRegemeが反転したら自動ドテン
    - 損切りはクールダウン無し（同バーで再エントリー可）
    """
    cols_keep = [time_col, "C_ALT", "V_ALT", "Trade", "acf_gate", "Regeme_vwap"]
    d = df_in[cols_keep].rename(columns={"C_ALT":"C","V_ALT":"V"}).copy()
    d = d.sort_values(time_col).reset_index(drop=True)

    n = len(d)
    side = np.zeros(n, dtype=int)
    pos_qty = np.zeros(n)
    entry_price = np.full(n, np.nan)
    trade_qty = np.zeros(n)
    trade_notional = np.zeros(n)
    fee_paid = np.zeros(n)
    fee_cum = np.zeros(n)
    realized_pnl = np.zeros(n)
    realized_pnl_cum = np.zeros(n)
    unrealized_pnl = np.zeros(n)
    equity = np.zeros(n)

    # MAE（含み損の最悪%）追跡 & per-trade最大損失
    mae_pct_cur_trade = 0.0
    max_adverse_pct_global = 0.0  # 負値（例: -3%）を更新
    max_adverse_ts = None

    worst_trade_realized = 0.0    # 最小の実現損益（負値が大きいほど悪い）

    # 損切りトラッキング
    stop_count = 0
    worst_stop_pnl = 0.0

    # 現在の建玉
    cur_qty = 0.0
    cur_entry = np.nan
    cur_fee_cum = 0.0
    cur_realized = 0.0

    for i in range(n):
        price = float(d.at[i, "C"])    
        gate = bool(d.at[i, "acf_gate"])  # 現在のACFゲート
        reg = int(d.at[i, "Regeme_vwap"]) # 1/0
        sig_bar = d.at[i, "Trade"]        # 1/0/NaN（ゲート中は常時方向を示す）

        # --- 損切りチェック（クールダウン無し） ---
        just_stopped = False
        if stop_loss_pct is not None and stop_loss_pct > 0 and cur_qty != 0.0:
            if cur_qty > 0:
                adverse_pct_now = (price / cur_entry) - 1.0
            else:
                adverse_pct_now = (cur_entry / price) - 1.0
            if adverse_pct_now <= -abs(stop_loss_pct):
                # 強制クローズ（損切り）
                close_qty = -cur_qty
                pnl_close = (price - cur_entry) * cur_qty
                cur_realized += pnl_close
                close_notional = abs(close_qty) * price
                fee_close = fee * close_notional
                cur_fee_cum += fee_close

                trade_qty[i] += close_qty
                trade_notional[i] += close_notional
                fee_paid[i] += fee_close
                realized_pnl[i] += pnl_close

                # ストップカウント＆ワースト更新
                stop_count += 1
                if pnl_close < worst_stop_pnl:
                    worst_stop_pnl = pnl_close

                # MAEリセット＆ポジション解消
                mae_pct_cur_trade = 0.0
                cur_qty = 0.0
                cur_entry = np.nan
                just_stopped = True

        # 望ましいサイド（ゲート中のみ方向が定義される）
        desired_side = 0
        if pd.notna(sig_bar):
            desired_side = 1 if int(sig_bar) == 1 else -1

        # --- Exit on ACF drop ---
        # ACFゲート外れ & オプションON ならフラット化
        force_exit = False
        if exit_on_acf_drop and (not gate) and (cur_qty != 0.0):
            force_exit = True

        # --- 取引判定 ---
        do_trade = False
        new_open_side = 0

        if force_exit:
            do_trade = True
            new_open_side = 0  # フラット
        else:
            if cur_qty == 0.0:
                # 未保有：ゲート中なら現在Regeme方向で新規
                if desired_side != 0:
                    do_trade = True
                    new_open_side = desired_side
            else:
                cur_side = 1 if cur_qty > 0 else -1
                # ドテン条件：ゲート中にRegeme反転（= desired_sideが現在と逆）
                if desired_side != 0 and desired_side == -cur_side:
                    do_trade = True
                    new_open_side = desired_side
                # 維持：その他の場合は何もしない

        # --- 約定処理 ---
        if do_trade:
            # 既存クローズ
            if cur_qty != 0.0:
                close_qty = -cur_qty
                pnl_close = (price - cur_entry) * cur_qty
                cur_realized += pnl_close
                close_notional = abs(close_qty) * price
                fee_close = fee * close_notional
                cur_fee_cum += fee_close

                trade_qty[i] += close_qty
                trade_notional[i] += close_notional
                fee_paid[i] += fee_close
                realized_pnl[i] += pnl_close

                # ワントレードの最大実現損益更新（悪い方を記録）
                if pnl_close < worst_trade_realized:
                    worst_trade_realized = pnl_close

                # クローズ後にMAEトラッキングをリセット
                mae_pct_cur_trade = 0.0
                cur_qty = 0.0
                cur_entry = np.nan

            # 新規（new_open_side != 0 のときのみ）
            if new_open_side != 0:
                target_qty = notional_size / price
                if min_qty and min_qty > 0:
                    target_qty = np.floor(target_qty / min_qty) * min_qty
                if target_qty > 0:
                    open_qty = new_open_side * target_qty
                    open_notional = abs(open_qty) * price
                    fee_open = fee * open_notional
                    cur_fee_cum += fee_open

                    trade_qty[i] += open_qty
                    trade_notional[i] += open_notional
                    fee_paid[i] += fee_open

                    cur_qty = open_qty
                    cur_entry = price

        # --- 評価 & MAE更新 ---
        if cur_qty != 0.0:
            u_pnl = (price - cur_entry) * cur_qty
        else:
            u_pnl = 0.0
        unrealized_pnl[i] = u_pnl

        # MAE（エントリー価格からの最大含み損%）を更新
        if cur_qty != 0.0:
            if cur_qty > 0:  # ロング
                adverse_pct = (price / cur_entry) - 1.0  # ロングに不利だと負値
            else:            # ショート
                adverse_pct = (cur_entry / price) - 1.0  # ショートに不利だと負値

            # 現トレードの最悪%（より負の方向へ）
            mae_pct_cur_trade = min(mae_pct_cur_trade, adverse_pct)

            # グローバル最悪を更新
            if mae_pct_cur_trade < max_adverse_pct_global:
                max_adverse_pct_global = mae_pct_cur_trade
                max_adverse_ts = d.at[i, time_col]

        # エクイティ
        if i > 0:
            realized_pnl_cum[i] = realized_pnl_cum[i-1] + realized_pnl[i]
            fee_cum[i] = fee_cum[i-1] + fee_paid[i]
        else:
            realized_pnl_cum[i] = realized_pnl[i]
            fee_cum[i] = fee_paid[i]

        equity[i] = initial_capital + realized_pnl_cum[i] + unrealized_pnl[i] - fee_cum[i]

        side[i] = 0 if cur_qty == 0 else (1 if cur_qty > 0 else -1)
        pos_qty[i] = cur_qty
        entry_price[i] = cur_entry

    out = d.copy()
    out["side"] = side
    out["position_qty"] = pos_qty
    out["entry_price"] = entry_price
    out["trade_qty"] = trade_qty
    out["trade_notional"] = trade_notional
    out["fee_paid"] = fee_paid
    out["fee_paid_cum"] = fee_cum
    out["realized_pnl"] = realized_pnl
    out["realized_pnl_cum"] = realized_pnl_cum
    out["unrealized_pnl"] = unrealized_pnl
    out["equity"] = equity

    # 追加メトリクス
    out.attrs["max_adverse_pct_global"] = max_adverse_pct_global * 100.0  # %
    out.attrs["max_adverse_ts"] = max_adverse_ts
    out.attrs["worst_trade_realized"] = worst_trade_realized  # USD
    out.attrs["stop_count"] = stop_count
    out.attrs["worst_stop_pnl"] = worst_stop_pnl

    return out

# 6) 実行：フィルター無し vs ACFゲート有り  ------------------------------
df_bt_A = df_all[[time_col,"C_ALT","V_ALT","Trade_vwap_only","acf_gate","Regeme_vwap"]].rename(columns={"Trade_vwap_only":"Trade"})
df_bt_B = df_all[[time_col,"C_ALT","V_ALT","Trade","acf_gate","Regeme_vwap"]]

bt_A = run_backtest(df_bt_A, INITIAL_CAPITAL, TAKER_FEE, NOTIONAL_SIZE, MIN_QTY, EXIT_ON_ACF_DROP, STOP_LOSS_PCT)
bt_B = run_backtest(df_bt_B, INITIAL_CAPITAL, TAKER_FEE, NOTIONAL_SIZE, MIN_QTY, EXIT_ON_ACF_DROP, STOP_LOSS_PCT)

# 7) Summary & Plot  -------------------------------------------------------

def summary(bt: pd.DataFrame, tag: str):
    total_fee = bt['fee_paid'].sum()
    trade_count_bars = (bt['trade_qty'] != 0).sum()
    total_notional = bt['trade_notional'].sum()

    max_adverse_pct = bt.attrs.get("max_adverse_pct_global", 0.0)
    max_adverse_ts = bt.attrs.get("max_adverse_ts", None)
    worst_trade_realized = bt.attrs.get("worst_trade_realized", 0.0)
    stop_count = bt.attrs.get("stop_count", 0)
    worst_stop_pnl = bt.attrs.get("worst_stop_pnl", 0.0)

    print(f"=== {tag} ===")
    print(f"Total fee paid              : {total_fee:.2f} USD")
    print(f"Number of trade bars        : {trade_count_bars}")
    print(f"Total trade notional        : {total_notional:.2f} USD")
    print(f"Final equity                : {bt['equity'].iloc[-1]:.2f} USD")
    print(f"Max adverse excursion (MAE) : {max_adverse_pct:.2f}% at {max_adverse_ts}")
    print(f"Worst single-trade PnL      : {worst_trade_realized:.2f} USD")
    print(f"Stop-loss count             : {stop_count}")
    print(f"Worst stop PnL              : {worst_stop_pnl:.2f} USD")


summary(bt_A, "VWAP only (ref)")
summary(bt_B, "VWAP + ACF gate")

fig = go.Figure()
fig.add_trace(go.Scatter(x=bt_A[time_col], y=bt_A['equity'], mode='lines', name='VWAP only (ref)'))
fig.add_trace(go.Scatter(x=bt_B[time_col], y=bt_B['equity'], mode='lines', name='VWAP + ACF gate'))
fig.update_layout(title="Equity Curve Comparison", xaxis_title="Time", yaxis_title="Equity (USD)",
                  hovermode="x unified", template="plotly_white", height=520, width=1180)
fig.show()


=== VWAP only (ref) ===
Total fee paid              : 252.04 USD
Number of trade bars        : 720
Total trade notional        : 720107.31 USD
Final equity                : 107.73 USD
Max adverse excursion (MAE) : -4.83% at 2025-09-19 14:06:00
Worst single-trade PnL      : -25.29 USD
Stop-loss count             : 1
Worst stop PnL              : -25.29 USD
=== VWAP + ACF gate ===
Total fee paid              : 2.45 USD
Number of trade bars        : 8
Total trade notional        : 6996.41 USD
Final equity                : 1564.98 USD
Max adverse excursion (MAE) : -4.49% at 2025-09-23 15:00:00
Worst single-trade PnL      : 0.00 USD
Stop-loss count             : 1
Worst stop PnL              : -31.38 USD


In [35]:
# 0) Libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1) パラメータ
CSV_FILE = "Market_Bybit_ASTERUSDT_1min_20250901-20250926.csv"
time_col = "timestamp"
ACF_WINDOW = 5       # rolling window (bars)
SMOOTH_HALFLIFE = 5  # ACFのEWMA平滑（必要なら）

# 2) データ
df = pd.read_csv(f'01.data/{CSV_FILE}')
df[time_col] = pd.to_datetime(df[time_col])
df = df.sort_values(time_col).reset_index(drop=True)

# 3) ACF(lag=1) のローリング計算（pandasのrolling相関で安定）
df["ret"] = np.log(df["C"]).diff()
acf1 = df["ret"].rolling(ACF_WINDOW, min_periods=ACF_WINDOW).corr(df["ret"].shift(1))
# 平滑化（任意）
acf1 = acf1.ewm(halflife=SMOOTH_HALFLIFE, adjust=False).mean()
df["acf_lag1"] = acf1

# 4) 可視化（ズーム共有・range-slider無効）
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,  # ← ズーム連動
    row_heights=[0.7, 0.3],
    vertical_spacing=0.05,
    subplot_titles=("Candlestick", "ACF (lag=1)")
)

# 上段：ローソク足
fig.add_trace(
    go.Candlestick(
        x=df[time_col],
        open=df["O"], high=df["H"], low=df["L"], close=df["C"],
        name="Candles"
    ), row=1, col=1
)

# 下段：ACF折れ線
fig.add_trace(
    go.Scatter(
        x=df[time_col], y=df["acf_lag1"],
        mode="lines", name="ACF lag1"
    ), row=2, col=1
)

# 重要：rangesliderを無効化（これが崩れの原因）
fig.update_layout(
    xaxis_rangeslider_visible=False,   # 上段x軸
    title="Candlestick + Rolling ACF (linked zoom, no rangeslider)",
    template="plotly_white",
    height=720, width=1200
)

# 補助線：ACFのゼロライン
fig.add_hline(y=0, line_width=1, line_dash="dot", row=2, col=1)

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_xaxes(title_text="Time",  row=2, col=1)
fig.update_yaxes(title_text="ACF",   row=2, col=1)

fig.show()
